In [ ]:
# for dev purpose - get username and password
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, snpQueries
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating disability_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
last_month = last_date_of_last_month_str[:7]
reporting_year_value=last_date_of_last_month.year
print(reporting_year_value)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
#TODO: disability_source1 can be removed
#TODO: lets QA here to make sure everything works 
#TODO: save off raw pulls
if new_pull:
    pull_disability_bq = snpQueries.pull_disability_bq(reporting_year_value)
    disability_df = pull_disability_bq[pull_disability_bq['Disability_Ind']=='Yes'][['Member_ID']]
    disability_df["disability_ind"] = True
    disability_df.to_parquet(f"{output_path}/disability_df.parquet")
else:
    try:
        disability_df = pd.read_parquet(f"{output_path}/disability_df.parquet")

    except:
        pull_disability_bq = snpQueries.pull_disability_bq(reporting_year_value)
        disability_df = pull_disability_bq[pull_disability_bq['Disability_Ind']=='Yes'][['Member_ID']]
        disability_df["disability_ind"] = True
        disability_df.to_parquet(f"{output_path}/disability_df.parquet")

In [ ]:
# output the top rows and size
print(disability_df.shape)
print(disability_df.head(3))

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))